## 2.1 Optimizing algorithms

**Learning Objectvies**

- Remember different optimization methods such as (Stochastic) Gradient Descent, Momentum, RMSProp and Adam
- Use random minibatches to accelerate the convergence and improve the optimization
- Know the benefits of learning rate decay and apply it to your optimization

### 2.1.1 Mini-batch gradient descent

- Split up your training set into smaller training sets called mini-batches

### 2.1.2 Understanding mini-batch gradient descent

**Training with mini batch gradient descent**

<img src="images/2.1.2 Training with mini batch gradient descent.png">

- With batch gradient descent on every iteration, you go through the entire training set and expect the cost to go down on every single iteration. So if we plot the cost function J as a function of different iterations, it should decrease on every single iteration. And if it goes up even on one iteration then something is wrong due to learning rate being too big or other reasons.
- On mini-batch gradient descent, it may not decrease on every iteration. In perticular, on every iteration you're processing some X{t}, Y{t} and if you plot the cost function J{t} which is computed using just X{t}, Y{t}, then as if on every iteration you're tranining on a different training set or really training on a different mini batch. The plot should trend downwards but with a little noise. The reason is that maybe X{1}, Y{1} is just the rows of easy mini batch so your cost might be a bit lower, but then maybe just by chance X{2}, Y{2} is just a harder mini batch like you have some mislabled examples in it, in which caused the cost a bit higher and so on.That's why you get those oscillations as you plot the cost when you're running mini batch gradient descent.

**Choosing your mini-batch size**

- if you choose size = m, then it becomes batch gradient descent which starts from somewhere on the contours and be able to take relatively low noise and large steps then keep matching to the minimum.
- if you choose size = 1, then it's called stochastic gradient descent which in contrast can be extremely noisy and on average it'll take you in a good direction but sometimes head to the wrong directions as well.The stochastic gradient descent won't ever converge and it'll always just oscillate and wander around the region of the minimum but won't ever just head to the minimum and stay there.

So here is the guidelines:

- small training set (<= 2000 examples) ? use batch gradient descent
- mini-bath: choose size which is a power of 2, like 64, 128, 256, 512. Also make sure all of your X{t}, Y{t} that fits in CPU/GPU memory.

### 2.1.3 Exponentially weighted averages (EWA)

Formular: $v_{t} = \beta v_{t-1} + (1 - \beta)\theta_{t}$

<img src="images\2.1.3 Exponentially weighted averages.png">

- red line with $\beta = 0.9$ (1/(1-0.9) = 10 days EWA)
- green line with $\beta = 0.98$ (1/(1-0.98) 50 days EWA)
- yellow line with $\beta = 0.5$ (1/(1-0.5) 2 days EWA)

### 2.1.4 Understanding exponentially weighted averages


**Implementing EWA**



### 2.1.5 Bias correction in EWA

<img src="images/2.1.5 Bias correction in EWA.png">

If you implement the formula above using $\beta$=0.98, you won't get the green curve but the purple curve which starts off really low. This is because you start with $v_{0}=0$ which make the first few day's estimate inaccuracy.Instead of using $v_{t}$, we use $\frac{v_{t}}{1-\beta^{t}}$

### 2.1.6 Gradient descent with momentum

The basic idea is to compute an EWA of your gradients then use that gradient to update your weights instead.

<img src="images/2.1.6 Gradient descent with momentum.png">

**Implementation details**:

On iteration t: 

- compute the usual derivatives dW, db on the current mini-batch/batch
- compute $v_{dW} = \beta v_{dW} + (1 - \beta)d_{W}$
- compute $v_{db} = \beta v_{db} + (1 - \beta)d_{b}$
- update your weight $W := W - \alpha v_{dW}$, $b := b - \alpha v_{db}$

Hyperparameters:  

- $\alpha$: learning rate
- $\beta$: control your EWA. The most common value is 0.9 which average the last 10 iterations gradients

**Note**: Here we don't use bias correction as after 10 iterations, your moving average will have warmed up and is no longer a bias estimate.

The above steps are to smooth out the steps of gradient descent. That's to say if you average out these gradients, you will find that the oscillations in the vertical direction will tend to average out to something closer to zero. So in the vertical direction where you want to slow things down, this will average out positive and negative numbers, so the average will be close to zero. While in the horizontal direction, all the derivatives are pointing to the right of the horizontal direction, so the average in the horizontal direction will still be pretty big. So with the algorithm, you'll find the gradient with momentum ends up eventually just taking steps that are much smaller oscilliations in the vertical direction but are more directed to just moving quickly in the horizontal direction. This allow our algorithm to take a more straightforward path to the minimum.

### 2.1.7 Root mean square prop (RMSprop)

The purpose of this algorithm is to slowdown the learning in the vertical direction and speed up learning or at least not slow it down in the horizontal direction.

**Implementation details**:

On iteration t: 

- compute the usual derivatives dW, db on the current mini-batch/batch
- compute $S_{dW} = \beta S_{dW} + (1 - \beta)d_{W^2}$ (sqaure here is an element-wise squaring operation. What this is doing is to keep an EWA of the squares of the derivatives)
- compute $S_{db} = \beta S_{db} + (1 - \beta)d_{b^2}$
- update $W:=W - \alpha \frac{dW}{\rootsqure{S_{dW}}$$

### 2.1.8 Adaptive Momentum optimization altorithm

It's a combination of momentum and RMS prop together.

**Implementation details**:

**Initialize** $V_{dw} = 0, S_{dW}=0, V_{db}=0, S_{db}=0$

On iteration t:

- Compute the derivatives of $d_{W}, d_{b}$ using current mini-batch gradient descent
- Do the momentum EWA, so \
  $V_{dw}=\beta_{1} V_{dW} + (1-\beta_{1})d_{W}$\ 
  $V_{db}=\beta_{1} V_{db} + (1-\beta_{1})d_{b}$\
  $S_{dW}=\beta_{2} S_{dW} + (1 - \beta_{2})d_{W^2}$\
  $S_{db}=\beta_{2} S_{db} + (1 - \beta_{2})d_{b^2}$
- Implement bias correction: 

<img src="images\2.1.8 Adam optimization altorithm1.png">
<img src="images\2.1.8 Adam optimization altorithm2.png">

### 2.1.9 Learning rate decay

**Why do you want to implement learning rate decay**

The intuition behind slowly reducing alpha is that maybe during the initial steps of learning, you could afford to take much bigger steps. But then as learning approaches converges, then having a slower learning rate allows you to take smaller steps.

**How to implement?**

<img src="images/2.1.9 Learning rate decay1.png">
<img src="images/2.1.9 Learning rate decay2.png">

### 2.1.10 The problem of local optima

<img src="images/2.1.10 The problem of local optima1.png">
<img src="images/2.1.10 The problem of local optima2.png">

## 2.2 Quiz

<img src="images/2.2.1.png">
1st
<img src="images/2.2.2.png">
2nd
<img src="images/2.2.3.png">
2nd & 4th
<img src="images/2.2.4.png">
1st
<img src="images/2.2.5.png">
4th
<img src="images/2.2.6.png">
2nd
<img src="images/2.2.7.png">
2nd & 3rd
<img src="images/2.2.8.png">
<img src="images/2.2.9.png">
2rd & 3rd & 4th & 5th
<img src="images/2.2.10.png">
3rd

## 2.3 Programming assignment

By completing this assignment you will:

- Understand the intuition between Adam and RMS prop

- Recognize the importance of mini-batch gradient descent

- Learn the effects of momentum on the overall performance of your model
